In [1]:
#my modules
import wrangle as wr
import explore as ex
import model as m

#Imports
import pandas as pd
import numpy as np

import nltk
# nltk.download('words')
# words = set(nltk.corpus.words.words())

imports loaded successfully, awaiting commands...


In [2]:
df = pd.read_parquet('df_prep.parquet')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246736 entries, 0 to 1246735
Data columns (total 8 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   date_received                 1246736 non-null  datetime64[ns]
 1   company_name                  1246736 non-null  object        
 2   state                         1246736 non-null  object        
 3   tags                          1246736 non-null  object        
 4   company_response_to_consumer  1246736 non-null  object        
 5   product_bins                  1246736 non-null  object        
 6   clean                         1246736 non-null  object        
 7   lemon                         1246736 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 76.1+ MB


In [4]:
relief = ['Closed with monetary relief', 'Closed with non-monetary relief']
no_relief = ['Closed with explanation']
df = df[df['company_response_to_consumer'] != "Untimely response"]
df = df[df['company_response_to_consumer'] != "Closed"]
df['response'] = np.where(df['company_response_to_consumer'].isin(relief),'relief','')
df['response'] = np.where(df['company_response_to_consumer'].isin(no_relief),'no_relief',df['response'])
df = df.drop(columns='company_response_to_consumer')

In [5]:
# split
train, validate, test = wr.split_data(df, 'response')

Prepared DF: (1238536, 8)
Train: (743121, 8)
Validate: (247707, 8)
Test: (247708, 8)


In [6]:
sentiment_df= wr.sentiment_analysis(train)

In [9]:
grouped_df = sentiment_df.groupby(['product_bins', 'response'])


In [10]:
result = grouped_df['sentiment'].mean()
result

product_bins     response 
bank             no_relief   -0.064332
                 relief      -0.025203
credit_card      no_relief    0.228293
                 relief       0.203661
credit_report    no_relief    0.127576
                 relief       0.093646
debt_collection  no_relief   -0.154207
                 relief      -0.148750
loans            no_relief    0.168811
                 relief       0.186377
money_service    no_relief    0.016596
                 relief      -0.015807
mortgage         no_relief    0.136263
                 relief       0.190315
Name: sentiment, dtype: float64